In [4]:
DAY = "21"

In [5]:
import os
with open(f'session.secret','r') as f:
    SESSION = f"session={f.read()}"

INPUT_FILE = f"{DAY}.txt"

In [6]:
# Manual INPUT
INPUT ="""INPUT"""

In [7]:
# Automatic INPUT
import os.path
if not os.path.isfile(INPUT_FILE):
    !curl https://adventofcode.com/2024/day/{DAY}/input --cookie {SESSION} > {INPUT_FILE}
with open(INPUT_FILE,'r') as f:
    INPUT = f.read().strip()

## Usefull import

In [8]:
import numpy as np
import pandas as pd
import networkx as nx
import copy as cp
import re

In [9]:
DIR = {(0,1):'>',(1,0):'v',(0,-1):'<',(-1,0):'^'}
DIRd = {v:k for k,v in DIR.items()}

## Solution

In [19]:
codes = INPUT.split()

In [12]:
NUMPAD_POS = {
    'A':(0,0)
    ,'0':(0,-1)
    ,'1':(-1,-2)
    ,'2':(-1,-1)
    ,'3':(-1,0)
    ,'4':(-2,-2)
    ,'5':(-2,-1)
    ,'6':(-2,0)
    ,'7':(-3,-2)
    ,'8':(-3,-1)
    ,'9':(-3,0)
}
NUMPAD_KEY = {v:k for k,v in NUMPAD_POS.items()}    

In [13]:
DIR_POS ={
    'A':(0,0),
    '^':(0,-1),
    '<':(1,-2),
    'v':(1,-1),
    '>':(1,0)
}
DIR_KEY = { v:k for k,v in DIR_POS.items()}

In [17]:
def movekey(direction,current_key,numpad=False):
    POS = NUMPAD_POS if numpad else DIR_POS
    KEY = NUMPAD_KEY if numpad else DIR_KEY
    
    cx,cy = POS[current_key]
    dx,dy = DIRd[direction]
    nx,ny = cx+dx,cy+dy
    
    if (nx,ny) not in KEY:
        return False
    return KEY[(nx,ny)]
    

In [18]:
G = nx.DiGraph()
for r1 in DIR_POS:
    for r2 in DIR_POS:
        for r3 in NUMPAD_POS:
            
            node = (r1,r2,r3)
            for user_key in DIR_POS:
                if user_key != 'A':
                    nr1=movekey(user_key,r1)
                    if nr1:
                        G.add_edge(node, (nr1,r2,r3), user_key=user_key)
                    continue
                if r1!='A':
                    nr2=movekey(r1,r2)
                    if nr2:
                        G.add_edge(node, (r1,nr2,r3), user_key=user_key)
                    continue
                
                if r2!='A':
                    nr3=movekey(r2,r3,numpad=True)
                    if nr3:
                        G.add_edge(node, (r1,r2,nr3), user_key=user_key)
                    continue
                        

In [20]:
def path_len(code):
    PRESS_A_LEN = 1
    current_node= ('A','A','A')
    total_len = 0
    for digit in code:
        next_node = ('A','A',digit)
        len_to_go = nx.shortest_path_length(G,current_node,next_node)
        total_len+=len_to_go+PRESS_A_LEN
        current_node=next_node
    return total_len

In [21]:
sum([int(c.replace('A',''))*path_len(c) for c in codes])

164960

## Part 2

In [25]:
def is_path_valid(s,e,path,numpad=False):
    c = s
    for d in path:
        c=movekey(d,c,numpad=numpad)
        if not c:
            return False
    return True

NUMPAD_SHORTEST=dict()
for s,(sx,sy) in NUMPAD_POS.items():
    for e,(ex,ey) in NUMPAD_POS.items():
        dy = (ey-sy)
        y = ('>' if dy>0 else '<')*abs(dy)
        dx = ex-sx
        x = ('v' if dx>0 else '^')*abs(dx)
        
        paths= [p for p in list(set([x+y,y+x])) if is_path_valid(s,e,p,numpad=True)]
        NUMPAD_SHORTEST[(s,e)] = [p+'A' for p in paths ]



In [26]:
DIRPAD_SHORTEST=dict()
for s,(sx,sy) in DIR_POS.items():
    for e,(ex,ey) in DIR_POS.items():
        dy = (ey-sy)
        y = ('>' if dy>0 else '<')*abs(dy)
        dx = ex-sx
        x = ('v' if dx>0 else '^')*abs(dx)
        
        paths=  [p for p in list(set([x+y,y+x])) if is_path_valid(s,e,p,numpad=False)]
        DIRPAD_SHORTEST[(s,e)]= [p+'A' for p in paths ]

In [34]:
DIRPAD_SHORTEST.update(NUMPAD_SHORTEST)

In [57]:
cache = dict()
def solve_shortest(n,code):
    if (n,code) in cache:
        return cache[(n,code)]
    
    if n==0:
        return len(code)
    
    total = 0
    for s,e in zip('A'+code,code):
        all_possibilities = DIRPAD_SHORTEST[(s,e)]
        total += min([solve_shortest(n-1,p) for p in all_possibilities])
    
    cache[(n,code)]=total
    return total

In [59]:
%%timeit
sum([solve_shortest(26,code)*int(code.replace('A','')) for code in codes])

3.42 µs ± 184 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
